# 2222222.ipynb

## What's new:

1- Without RSI

2- add labels only start and end of trends.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas_ta as ta


In [2]:
# 1- Load and Scaling Features
# Load and preprocess
df = pd.read_csv('datasets-28-111/XAGUSD-H1-rates.csv', sep='\t')

# Rename columns for easier access
df.rename(columns={
    '<DATE>': 'DATE',
    '<TIME>': 'TIME',
    '<OPEN>': 'OPEN',
    '<HIGH>': 'HIGH',
    '<LOW>': 'LOW',
    '<CLOSE>': 'CLOSE',
    '<TICKVOL>': 'TICKVOL',
    '<VOL>': 'VOL',
    '<SPREAD>': 'SPREAD'
}, inplace=True)

# Optional: Combine DATE and TIME into a single datetime column
df['DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])

# Drop rows with missing values
df.dropna(inplace=True)

# Sort data chronologically by DATETIME
df.sort_values(by='DATETIME', inplace=True)

# Reset index to ensure clean row order
df.reset_index(drop=True, inplace=True)

# Select features to scale
features = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']

# Apply MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

In [3]:
# 2- Label trend reversals (example: a simplistic method)
ZIGZAG_PCT = 2.0     # minimum % price move to define swing high/low (example: 2%)
ZIGZAG_DEVIATION = 5 # maximum number of bars allowed for backstep (like MT5)
ZIGZAG_DEPTH = 12    # minimum bars between peaks/troughs

# ===============================
# 2- Label trend reversals (using pandas-ta ZigZag)
# ===============================
# Apply ZigZag from pandas-ta
df['zigzag'] = ta.zigzag(
    df['HIGH'],
    df['LOW'],
    percent=ZIGZAG_PCT
)

# Generate labels:
# 0 = no signal, 1 = buy reversal, 2 = sell reversal
df['Label'] = 0

for i in range(1, len(df)):
    if pd.notna(df['zigzag'].iloc[i]) and pd.notna(df['zigzag'].iloc[i-1]):
        if df['zigzag'].iloc[i] > df['zigzag'].iloc[i-1]:
            df.loc[df.index[i], 'Label'] = 1  # Buy reversal
        elif df['zigzag'].iloc[i] < df['zigzag'].iloc[i-1]:
            df.loc[df.index[i], 'Label'] = 2  # Sell reversal



AttributeError: module 'pandas_ta' has no attribute 'zigzag'

In [ ]:
DEVIATION = 0.02        # Minimum relative price change (2% default)
BACKSTEP = 3            # Lookback steps to validate a swing

# ================================
# 2- ZigZag labeling function
# ================================
def zigzag_labels(high, low, close, deviation=0.02, backstep=3):
    """
    Generate reversal labels using a zigzag-like approach.
    Labels:
        0 = no signal
        1 = buy (swing low / up reversal)
        2 = sell (swing high / down reversal)
    """
    n = len(close)
    labels = np.zeros(n, dtype=int)
    last_pivot = 0
    last_pivot_type = 0  # 1 = high, -1 = low

    for i in range(backstep, n - backstep):
        # Check swing high
        if high[i] == max(high[i-backstep:i+backstep+1]):
            if last_pivot_type != 1 and (abs((high[i] - low[last_pivot]) / low[last_pivot]) >= deviation if last_pivot > 0 else True):
                labels[i] = 2  # Sell
                last_pivot = i
                last_pivot_type = 1

        # Check swing low
        if low[i] == min(low[i-backstep:i+backstep+1]):
            if last_pivot_type != -1 and (abs((high[last_pivot] - low[i]) / high[last_pivot]) >= deviation if last_pivot > 0 else True):
                labels[i] = 1  # Buy
                last_pivot = i
                last_pivot_type = -1

    return labels

# Apply zigzag labeling
df['Label'] = zigzag_labels(
    df['HIGH'].values,
    df['LOW'].values,
    df['CLOSE'].values,
    deviation=DEVIATION,
    backstep=BACKSTEP
)


In [ ]:
def plot_labeled_candles(df, n=150):
    """
    Plots the last n candles with BUY/SELL labels based on the 'Label' column.
    """
    # Use only the last n rows
    df_plot = df.tail(n).copy()
    df_plot['DATETIME'] = df_plot['DATE'] + ' ' + df_plot['TIME']

    # Plot the closing price
    plt.figure(figsize=(15, 6))
    plt.plot(df_plot['DATETIME'], df_plot['CLOSE'], label='Close Price', color='black', linewidth=1.5)

    # Plot BUY (label=1) and SELL (label=2) signals
    for idx, row in df_plot.iterrows():
        if row['Label'] == 1:
            plt.axvline(x=row['DATETIME'], color='green', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'BUY', color='green', ha='center', va='bottom', fontsize=9)
        elif row['Label'] == 2:
            plt.axvline(x=row['DATETIME'], color='red', linestyle='--', linewidth=1)
            plt.text(row['DATETIME'], row['CLOSE'], 'SELL', color='red', ha='center', va='top', fontsize=9)

    plt.title(f'Last {n} Candles with Trend Reversal Labels')
    plt.xlabel('Datetime')
    plt.ylabel('Close Price')
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
plot_labeled_candles(df)

# plot section